# 制作你自己的 **AI** 动作识别系统

设备：香橙派 Zero 3 （同时执行太多任务可能会导致设备卡顿）

## GNU/Linux 的基本操作

>GNU 是 "GNU's Not Unix!" 的递归缩写

`ctrl + alt + t` 打开终端

`ctrl + alt + F1-F7` 切换虚拟终端 tty1-7, 通常tty7号是图形界面

`ls` 查看此目录下的所有文件与文件夹，尝试 `ls -a`, `ls -l`, `ls -lh` （-h = --human-readable，人类可读）

`man` *manual* 或指令后面加 `--help` 查看指令说明书 

`pwd` *print working directory* 输出所在目录

`code` (装好vscode的前提下) 从此目录打开vscode






## 配置开发环境